#  "UJ SN2019 Zadanie 2: Nocne Ptasie Wędrówki"

# Convolutional neural network

## Train, validation and test data preparation

In [1]:
import pathlib
import numpy as np

SEED = 1024
np.random.seed(SEED)


def load_data(directory, dataset_name):
    p = pathlib.Path(directory)
    if not p.is_dir():
        raise ValueError(
            'Directory: {directory} does not exist. Please, run firstly data_preparation.ipynb for creating data')
    return np.load(pathlib.Path(directory + dataset_name + '.npy'))


def load_train_and_validation_data(data_dir):
    X_train = load_data(data_dir, 'X_train')
    y_train = load_data(data_dir, 'y_train')
    X_validation = load_data(data_dir, 'X_validation')
    y_validation = load_data(data_dir, 'y_validation')
    return X_train, y_train, X_validation, y_validation


data_directory = '../../data/'

In [2]:
from typing import Sequence

import torch
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(SEED)

BATCH_SIZE = 64

def get_train_and_validation_dataloaders(train_X, train_y, validation_X, validation_y, batch_size=64) -> Sequence[
    torch.utils.data.TensorDataset]:
    X_train: torch.Tensor = torch.from_numpy(train_X).float()
    X_validation: torch.Tensor = torch.from_numpy(validation_X).float()

    y_train: torch.Tensor = torch.from_numpy(train_y).float()
    y_validation: torch.Tensor = torch.from_numpy(validation_y).float()

    train_dataset = TensorDataset(X_train, y_train)
    validation_dataset = TensorDataset(X_validation, y_validation)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, validation_dataloader

In [3]:
train_X, train_y, validation_X, validation_y = load_train_and_validation_data(data_directory)
train_X = train_X.reshape(-1, train_X.shape[2], train_X.shape[3])
train_y = train_y.reshape(-1)
validation_X = validation_X.reshape(-1, validation_X.shape[2], validation_X.shape[3])
validation_y = validation_y.reshape(-1)
train_dataloader, validation_dataloader = get_train_and_validation_dataloaders(train_X, train_y, validation_X,
                                                                               validation_y)

#### Check count of each class in train labels

In [5]:
counter_one = 0
counter_zero = 0
for label in train_y:
    if label == 1:
        counter_one +=1
    else:
        counter_zero +=1
        
print(counter_zero)
print(counter_one)

28343
21587


## Convolutional neural network
Predict bird's voice existing probability in 0.1 s

In [7]:
class BirdDetector(torch.nn.Module):
    
    def __init__(self):
        super(BirdDetector, self).__init__()
        self.pool = torch.nn.MaxPool2d(2)
        
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=128, kernel_size=5, padding=2)
        self.bn1 = torch.nn.BatchNorm2d(128)
        
        self.conv2 = torch.nn.Conv2d(in_channels=128, out_channels=96, kernel_size=5, padding=2)
        self.bn2 = torch.nn.BatchNorm2d(96)
        
        self.conv3 = torch.nn.Conv2d(in_channels=96, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(64)
        
        self.fc1 = torch.nn.Linear(448, 100)
        self.fc2 = torch.nn.Linear(100, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
            
    def forward(self, x):
        
        out = self.pool(self.bn1(torch.relu(self.conv1(x))))
        out = self.pool(self.bn2(torch.relu(self.conv2(out))))
        out = self.pool(self.bn3(torch.relu(self.conv3(out))))
        
        out = out.view(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        
        return self.sigmoid(out)
    
model = BirdDetector()

#### Create directory for models if not exists

In [8]:
saved_models_directory = '../../saved_model/'
p = pathlib.Path(saved_models_directory)
if not p.is_dir():
    print(f'Creating directory: {saved_models_directory} as it does not exist')
    p.mkdir(parents=True, exist_ok=True)

## Training
I decided to use Adam optimizer with higher than default learning rate. <br>
It will be decreased because of using scheduler (ReduceLROnPlateau). If metric has stopped improving in following 3 epochs, then learning rate will be divided by 2.

In [9]:
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from tqdm import trange

optimizer: torch.optim.Optimizer = optim.Adam(model.parameters(), lr=0.008, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, 
                                                       patience=3, verbose=True)

criterion = torch.nn.BCELoss()
epoch: int = 100

best_validation_roc_auc = 0
epochs_without_improvement = 0
MAX_POSSIBLE_EPOCHS_WITHOUT_IMPROVEMENT = 30

predictions_from_the_best_model = None
predictions_from_the_best_model_validation = None

labels_train_from_the_best_model = None
labels_validation_from_the_best_model = None

for e in trange(epoch):
    print(f"EPOCH: {e}")
    
    preds_train = []
    labels_train = []
    correct_train: int = 0 
    loss_train : int = 0
    for i, (x, y) in enumerate(train_dataloader):
        correct_in_batch = 0
        optimizer.zero_grad()
        output: torch.Tensor = model(x.view(-1, 1, x.shape[1], x.shape[2]))
        preds_train.append(output.flatten())
        labels_train.append(y)
        loss: torch.Tensor = criterion(output.view(-1), y)
        loss.backward()
        optimizer.step()
        for predicted, correct in zip(output.view(-1), y): 
            if predicted >= 0.5 and correct==1:
                correct_in_batch += 1
            if predicted < 0.5 and correct==0:
                correct_in_batch += 1
        correct_train += correct_in_batch
        loss_train += loss.item()
    preds_train = torch.cat(preds_train)
    labels_train = torch.cat(labels_train)
    print(f"Train accuracy: {correct_train / len(train_X)}")
    print(f"Loss: {loss_train / len(train_X)}")
    
    with torch.no_grad():
        preds = []
        validation_y = []
        for i, (x, y) in enumerate(validation_dataloader):
            output: torch.Tensor = model(x.view(-1, 1, x.shape[1], x.shape[2]))
            preds.append(output.flatten())
            validation_y.append(y)
        preds = torch.cat(preds)
        validation_y = torch.cat(validation_y)
        score = roc_auc_score(validation_y.numpy(), preds.numpy())
        if score > best_validation_roc_auc:
            best_validation_roc_auc = score
            torch.save(model.state_dict(), '../../saved_model/model.pt')
            predictions_from_the_best_model = preds_train
            predictions_from_the_best_model_validation = preds
            labels_train_from_the_best_model = labels_train
            labels_validation_from_the_best_model = validation_y
            epochs_without_improvements = 0
        else:
            epochs_without_improvement += 1
            
        if epochs_without_improvement == MAX_POSSIBLE_EPOCHS_WITHOUT_IMPROVEMENT:
            break
        print(f"Validation ROC_AUC score: {score}\n")
        scheduler.step(score)

  0%|          | 0/100 [00:00<?, ?it/s]

EPOCH: 0
Train accuracy: 0.5496094532345284
Loss: 0.010985177101173072


  1%|          | 1/100 [07:42<12:43:54, 462.98s/it]

Validation ROC_AUC score: 0.5187027943467829

EPOCH: 1
Train accuracy: 0.5653715201281795
Loss: 0.010607576722399687


  2%|▏         | 2/100 [14:38<12:12:52, 448.70s/it]

...

Validation ROC_AUC score: 0.6747694373411823

EPOCH: 57
Train accuracy: 0.8200480672942119
Loss: 0.0056218991301710455


 58%|█████▊    | 58/100 [6:32:54<4:42:30, 403.60s/it]

Validation ROC_AUC score: 0.6737005217257366

EPOCH: 58
Train accuracy: 0.8258561986781494
Loss: 0.0055031281469549555


 59%|█████▉    | 59/100 [6:39:37<4:35:49, 403.64s/it]

Validation ROC_AUC score: 0.6768519954334861

Epoch    58: reducing learning rate of group 0 to 1.0000e-03.
EPOCH: 59
Train accuracy: 0.8315441618265572
Loss: 0.0053562847315003085


In [10]:
print(f"The highest validation roc auc: {best_validation_roc_auc}")

The highest validation roc auc: 0.685988128082064


## Prepare data for second model (classifier)

In [11]:
train_input_for_second_model = []
for i in range(0, len(predictions_from_the_best_model), 10):
    train_input_for_second_model.append(predictions_from_the_best_model[i : i+10].clone().detach())
    
train_labels_for_second_model = []
for i in range(0, len(labels_train_from_the_best_model), 10):
    train_labels_for_second_model.append(torch.max(labels_train_from_the_best_model[i: i+10]))
    
    
validation_input_for_second_model = []
for i in range(0, len(predictions_from_the_best_model_validation), 10):
    validation_input_for_second_model.append(predictions_from_the_best_model_validation[i : i+10].clone().detach())
    
validation_labels_for_second_model = []
for i in range(0, len(labels_validation_from_the_best_model), 10):
    validation_labels_for_second_model.append(torch.max(labels_validation_from_the_best_model[i: i+10]))
    

#### Check count of each class in train labels

In [12]:
counter_one = 0
counter_zero = 0
for label in train_labels_for_second_model:
    if label == 1:
        counter_one +=1
    else:
        counter_zero +=1
        
print(counter_zero)
print(counter_one)

2312
2681


In [13]:
batch_size=64
X_train_classifier: torch.Tensor = torch.stack(train_input_for_second_model)
X_validation_classifier: torch.Tensor = torch.stack(validation_input_for_second_model)
        
y_train_classifier: torch.Tensor = torch.stack(train_labels_for_second_model)
y_validation_classifier: torch.Tensor = torch.stack(validation_labels_for_second_model)

train_dataset_classifier = TensorDataset(X_train_classifier, y_train_classifier)
validation_dataset_classifier = TensorDataset(X_validation_classifier, y_validation_classifier)
    
train_dataloader_classifier = DataLoader(train_dataset_classifier, batch_size=batch_size, shuffle=True)
validation_dataloader_classifier = DataLoader(validation_dataset_classifier, batch_size=batch_size, shuffle=True)

In [14]:
data_for_classifier_directory = 'data/'
p = pathlib.Path(data_for_classifier_directory)
if not p.is_dir():
    print(f'Creating directory: {data_for_classifier_directory} as it does not exist')
    p.mkdir(parents=True, exist_ok=True)

In [15]:
torch.save(X_train_classifier, data_for_classifier_directory + 'X_train_classifier.pt')
torch.save(y_train_classifier, data_for_classifier_directory + 'y_train_classifier.pt')
torch.save(X_validation_classifier, data_for_classifier_directory + 'X_validation_classifier.pt')
torch.save(y_validation_classifier, data_for_classifier_directory + 'y_validation_classifier.pt')